Implement L2 regularization on cat-dog classification neural network. Train the model on the dataset, and observe the impact of the regularization on the weight parameters. (Do not use data augmentation).

a. L2 regularization using optimizer’s weight decay

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import PIL.Image as Image
import pandas as pd
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import AlexNet_Weights
from torchvision import transforms as T

In [3]:
EPOCHS = 10
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 100
LR = 0.001
WEIGHT_DECAY = 0.001
LOG_INTERVAL = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TRANSFORM = T.Compose([
    T.ToTensor(),
    T.Resize([224, 224]),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
CLASSES = {'dogs': 0, 'cats': 1}

In [4]:
def get_df(path, classes=['dogs', 'cats']):
    paths = pd.DataFrame({'class': [], 'path': []})
    for c in classes:
        df = pd.DataFrame({
            'class': c,
            'path': glob.glob(path + c + '/*')
        })

        paths = pd.concat([paths, df])

    paths.reset_index(inplace=False)

    return paths


class CatDogDataset(Dataset):
    def __init__(self, df, classes, transform=None):
        self.paths = df
        self.classes = classes
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        row = self.paths.iloc[idx]
        img = Image.open(row['path'])
        if self.transform is not None:
            return self.transform(img), self.classes[row['class']]
        else:
            return img, self.classes[row['class']]


def get_model():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', weights=AlexNet_Weights.DEFAULT)
    model.features.requires_grad = False
    model.classifier = nn.Sequential(
        *model.classifier[:-1],
        nn.Linear(4096, 2, bias=True)
    )
    return model

In [5]:
def train(model, train_loader, criterion, optimizer, device="cpu"):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * len(data)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(train_loader.dataset)
    acc = 100. * correct / len(train_loader.dataset)
    return acc, running_loss

def test(model, test_loader, criterion, device="cpu"):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            running_loss += criterion(output, target).item() * len(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    return acc, running_loss

In [6]:
train_dataset = CatDogDataset(get_df('./data/cats_and_dogs_filtered/train/'), CLASSES, TRANSFORM)
test_dataset = CatDogDataset(get_df('./data/cats_and_dogs_filtered/validation/'), CLASSES, TRANSFORM)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, shuffle=True)

In [7]:
modela = get_model()

modela = modela.to(DEVICE)

Using cache found in /home/vi-b-04/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
print(modela)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:
criterion = nn.CrossEntropyLoss()
optimizera = optim.SGD(modela.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

In [10]:
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch: {epoch}")

    print("\tTraining: ")
    train_acc, train_loss = train(modela, train_loader, criterion, optimizera, DEVICE)
    print("\t\tAccuracy: {:.4}%".format(train_acc))
    print("\t\tLoss: {:.4}".format(train_loss))

    print("\tValidation: ")
    test_acc, test_loss = test(modela, test_loader, criterion, DEVICE)
    print("\t\tAccuracy: {:.4}%".format(test_acc))
    print("\t\tLoss: {:.4}".format(test_loss))

Epoch: 1
	Training: 
		Accuracy: 90.05%
		Loss: 0.223
	Validation: 
		Accuracy: 96.7%
		Loss: 0.1052
Epoch: 2
	Training: 
		Accuracy: 95.75%
		Loss: 0.1159
	Validation: 
		Accuracy: 96.2%
		Loss: 0.09391
Epoch: 3
	Training: 
		Accuracy: 96.85%
		Loss: 0.084
	Validation: 
		Accuracy: 96.2%
		Loss: 0.08176
Epoch: 4
	Training: 
		Accuracy: 97.4%
		Loss: 0.07191
	Validation: 
		Accuracy: 96.7%
		Loss: 0.07449
Epoch: 5
	Training: 
		Accuracy: 97.3%
		Loss: 0.05976
	Validation: 
		Accuracy: 96.9%
		Loss: 0.07008
Epoch: 6
	Training: 
		Accuracy: 98.55%
		Loss: 0.04728
	Validation: 
		Accuracy: 96.4%
		Loss: 0.07575
Epoch: 7
	Training: 
		Accuracy: 98.6%
		Loss: 0.04313
	Validation: 
		Accuracy: 96.9%
		Loss: 0.0724
Epoch: 8
	Training: 
		Accuracy: 99.1%
		Loss: 0.03453
	Validation: 
		Accuracy: 96.1%
		Loss: 0.07368
Epoch: 9
	Training: 
		Accuracy: 98.95%
		Loss: 0.03342
	Validation: 
		Accuracy: 96.6%
		Loss: 0.07305
Epoch: 10
	Training: 
		Accuracy: 99.1%
		Loss: 0.0267
	Validation: 
		Accu

b. L2 regularization using loop to find L2 norm of weights

In [11]:
REGULARIZATION = 2

In [12]:
def train_with_regularization(model, train_loader, criterion, optimizer, regularization=2, lambda_val=0.001, device="cpu"):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        running_loss += loss.item() * len(data)
        norm = sum(torch.norm(p, regularization) for p in model.parameters())
        loss = loss + lambda_val * norm
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(train_loader.dataset)
    acc = 100. * correct / len(train_loader.dataset)
    return acc, running_loss

In [13]:
modelb = get_model()

modelb = modelb.to(DEVICE)

Using cache found in /home/vi-b-04/.cache/torch/hub/pytorch_vision_v0.10.0


In [14]:
print(modelb)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [15]:
optimizerb = optim.SGD(modelb.parameters(), lr=LR)

In [16]:
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch: {epoch}")

    print("\tTraining: ")
    train_acc, train_loss = train_with_regularization(modelb, train_loader, criterion, optimizerb, REGULARIZATION, WEIGHT_DECAY, DEVICE)
    print("\t\tAccuracy: {:.4}%".format(train_acc))
    print("\t\tLoss: {:.4}".format(train_loss))

    print("\tValidation: ")
    test_acc, test_loss = test(modelb, test_loader, criterion, DEVICE)
    print("\t\tAccuracy: {:.4}%".format(test_acc))
    print("\t\tLoss: {:.4}".format(test_loss))

Epoch: 1
	Training: 
		Accuracy: 92.05%
		Loss: 0.2129
	Validation: 
		Accuracy: 95.1%
		Loss: 0.121
Epoch: 2
	Training: 
		Accuracy: 95.95%
		Loss: 0.1086
	Validation: 
		Accuracy: 96.1%
		Loss: 0.09705
Epoch: 3
	Training: 
		Accuracy: 96.2%
		Loss: 0.09658
	Validation: 
		Accuracy: 96.1%
		Loss: 0.09116
Epoch: 4
	Training: 
		Accuracy: 97.35%
		Loss: 0.0726
	Validation: 
		Accuracy: 96.3%
		Loss: 0.08959
Epoch: 5
	Training: 
		Accuracy: 98.4%
		Loss: 0.0547
	Validation: 
		Accuracy: 96.2%
		Loss: 0.08438
Epoch: 6
	Training: 
		Accuracy: 98.15%
		Loss: 0.04709
	Validation: 
		Accuracy: 96.3%
		Loss: 0.08942
Epoch: 7
	Training: 
		Accuracy: 98.85%
		Loss: 0.03677
	Validation: 
		Accuracy: 96.4%
		Loss: 0.08241
Epoch: 8
	Training: 
		Accuracy: 99.1%
		Loss: 0.03135
	Validation: 
		Accuracy: 96.6%
		Loss: 0.08384
Epoch: 9
	Training: 
		Accuracy: 99.35%
		Loss: 0.02712
	Validation: 
		Accuracy: 96.5%
		Loss: 0.08213
Epoch: 10
	Training: 
		Accuracy: 99.45%
		Loss: 0.02413
	Validation: 
		